In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd

In [2]:
all_events = pd.read_csv('/content/drive/MyDrive/musicrecom/melodice/melodice_all_events.csv')

In [3]:
all_events

,game_id,link_id,idx,cookieid,event_date,event_type
0,242,3260,-1,0-0,2016-05-19 18:12:44.153372,EA
1,242,3261,-1,0-0,2016-05-19 18:18:04.419495,EA
2,242,3262,-1,0-0,2016-05-19 18:19:00.665417,EA
3,242,3263,-1,0-0,2016-05-19 18:19:48.678645,EA
4,242,3264,-1,0-0,2016-05-19 18:21:56.611650,EA
...,...,...,...,...,...,...
73245961,135,3042,32,1618059452-5089,2022-11-07 09:43:24.726066,VU
73245962,135,16308,34,1618059452-5089,2022-11-07 09:43:29.700739,VU
73245963,135,44543,40,1618059452-5089,2022-11-07 09:57:15.209875,VU
73245964,135,3036,47,1618059452-5089,2022-11-07 10:17:12.852845,VU


In [4]:
ea_events = all_events[all_events["event_type"] == 'EA'].drop_duplicates(["link_id", "cookieid"])
len(ea_events)

19210040

In [5]:
skipped_events = all_events[all_events["event_type"].isin(['EP', 'EN', 'EL'])].drop_duplicates(["link_id", "cookieid"])
len(skipped_events)

2131461

In [6]:
upvoted_events = all_events[all_events["event_type"] == 'VU'].drop_duplicates(["link_id", "cookieid"])
len(upvoted_events)

22464

In [7]:
downvoted_events = all_events[all_events["event_type"] == 'VD'].drop_duplicates(["link_id", "cookieid"])
len(downvoted_events)

13436

In [8]:
remove_vote_events = all_events[all_events["event_type"] == 'RM'].drop_duplicates(["link_id", "cookieid"])
len(remove_vote_events)

15540

In [9]:
ea_events = ea_events.groupby(['link_id']).agg({
    'cookieid': list,
}).reset_index()

In [10]:
skipped_events = skipped_events.groupby(['link_id']).agg({
    'cookieid': list,
}).reset_index()

In [11]:
upvoted_events = upvoted_events.groupby(['link_id']).agg({
    'cookieid': list,
}).reset_index()


In [12]:
downvoted_events = downvoted_events.groupby(['link_id']).agg({
    'cookieid': list,
}).reset_index()

In [13]:
remove_vote_events = remove_vote_events.groupby(['link_id']).agg({
    'cookieid': list,
}).reset_index()

In [14]:
ea_events["len"] = ea_events.cookieid.map(len)
ea_events = ea_events[ea_events["len"] > 1]

In [15]:
skipped_events["len"] = skipped_events.cookieid.map(len)
skipped_events = skipped_events[skipped_events["len"] > 1]

In [16]:
upvoted_events["len"] = upvoted_events.cookieid.map(len)
upvoted_events = upvoted_events[upvoted_events["len"] > 1]

In [17]:
downvoted_events["len"] = downvoted_events.cookieid.map(len)
downvoted_events = downvoted_events[downvoted_events["len"] > 1]

In [18]:
remove_vote_events["len"] = remove_vote_events.cookieid.map(len)
remove_vote_events = remove_vote_events[remove_vote_events["len"] > 1]

In [19]:
sentences = ea_events["cookieid"].tolist() + skipped_events["cookieid"].tolist() \
 + upvoted_events["cookieid"].tolist() + downvoted_events["cookieid"].tolist() + remove_vote_events["cookieid"].tolist()

In [20]:
len(sentences)

90189

In [22]:
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=sentences,
    vector_size=50,
    window=1000,
    min_count=0,
    workers=7
)

In [23]:
model.build_vocab(sentences, progress_per=10000)


In [24]:
model.train(sentences, total_examples=model.corpus_count, epochs=20, report_delay=1)


(423096120, 427337500)

In [25]:
from gensim.models import KeyedVectors
word_vectors = model.wv
word_vectors.save("/content/drive/MyDrive/musicrecom/wv/word2vec.wordvectors")